In [1]:
%load_ext autoreload
%autoreload 2

# ExpectationValue

In [2]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import PauliSumOp

observable = PauliSumOp.from_list([("XX", 1), ("YY", 2), ("ZZ", 3)])
print("observable\n", observable)

ansatz = RealAmplitudes(num_qubits=2, reps=2)
print("ansatz\n", ansatz)

observable
 1.0 * XX
+ 2.0 * YY
+ 3.0 * ZZ
ansatz
      ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘


In [3]:
from qiskit.quantum_info import Statevector

expval = Statevector(ansatz.bind_parameters([0, 1, 1, 2, 3, 5])).expectation_value(
    observable.primitive
)
print(expval.real)

1.8420921273153719


## ExpectationValue class

In [4]:
from qiskit.evaluators import PauliExpectationValue

### PauliExpectationValue

Evaluate the expectation value by sampling. This supports both AerSimulator and IBMQ backends.

In [5]:
from qiskit.providers.aer import AerSimulator

In [6]:
backend = AerSimulator()

expval = PauliExpectationValue(ansatz, observable, backend=backend)

expval.evaluate([0, 1, 1, 2, 3, 5], shots=3000)

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=array(1.81466667+0.j), variance=array(11.13181689+0.j), confidence_interval=[((1.7218892642251156+0j), (1.9074440691082177+0j))])

In [7]:
# no parameter case

circuit = ansatz.bind_parameters([0, 1, 1, 2, 3, 5])
expval = PauliExpectationValue(circuit, observable, backend=backend)
expval.evaluate()

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=array(1.796875+0.j), variance=array(11.24065399+0.j), confidence_interval=[((1.6360011012920497+0j), (1.9577488987079503+0j))])

### Exact simulation by SaveExpectationValueVariance

In [8]:
from qiskit.evaluators import ExactExpectationValue

In [9]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
%time expval.evaluate([0, 1, 1, 2, 3, 5])

CPU times: user 42 ms, sys: 9.91 ms, total: 51.9 ms
Wall time: 6.58 ms


ExpectationValueResult(value=array(1.84209213), variance=array(6.43276352), confidence_interval=None)

In [10]:
%time expval.evaluate([1, 1, 2, 3, 5, 8])

CPU times: user 2.09 ms, sys: 1.24 ms, total: 3.32 ms
Wall time: 2.54 ms


ExpectationValueResult(value=array(0.15611897), variance=array(5.35681431), confidence_interval=None)

### Transpile options and Run options

In [11]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
# setter
expval.set_transpile_options(optimization_level=2)
expval.set_run_options(shots=1_000_000)
expval.evaluate([0, 1, 1, 2, 3, 5])

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=array(1.842808+0.j), variance=array(11.13573117+0.j), confidence_interval=[((1.8377203333117815+0j), (1.847895666688219+0j))])

In [12]:
# Method chain
expval.set_run_options(shots=8192).evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=array(1.84594727+0.j), variance=array(11.0951001+0.j), confidence_interval=[((1.7899597994185126+0j), (1.9019347318314874+0j))])

In [13]:
# evaluate's option
expval.evaluate([0, 1, 1, 2, 3, 5], shots=300)

ExpectationValueResult(value=array(1.9+0.j), variance=array(11.04884444+0.j), confidence_interval=[((1.6085270551649122+0j), (2.191472944835088+0j))])

In [14]:
from qiskit.test.mock import FakeBogota

backend = AerSimulator.from_backend(FakeBogota())

expval = PauliExpectationValue(ansatz, observable, backend=backend)
# expval.set_transpile_options(initial_layout=[1, 2])
for circ in expval.transpiled_circuits:
    print(circ)
    print(circ.metadata)
expval.evaluate([0, 1, 1, 2, 3, 5])

global phase: π
               ┌────┐┌──────────────┐┌────┐┌────────┐     ┌────┐»
      q_0 -> 0 ┤ √X ├┤ Rz(θ[0] + π) ├┤ √X ├┤ Rz(3π) ├──■──┤ √X ├»
               ├────┤├──────────────┤├────┤├────────┤┌─┴─┐├────┤»
      q_1 -> 1 ┤ √X ├┤ Rz(θ[1] + π) ├┤ √X ├┤ Rz(3π) ├┤ X ├┤ √X ├»
               └────┘└──────────────┘└────┘└────────┘└───┘└────┘»
ancilla_0 -> 2 ─────────────────────────────────────────────────»
                                                                »
ancilla_1 -> 3 ─────────────────────────────────────────────────»
                                                                »
ancilla_2 -> 4 ─────────────────────────────────────────────────»
                                                                »
         c9: 2/═════════════════════════════════════════════════»
                                                                »
«               ┌──────────────┐┌────┐┌────────┐     ┌────┐┌──────────────┐»
«      q_0 -> 0 ┤ Rz(θ[2] + π) ├┤ √X ├┤ Rz(3π) ├─

ExpectationValueResult(value=array(1.55395508+0.j), variance=array(11.98341149+0.j), confidence_interval=[((1.4941127030665997+0j), (1.6137974531834003+0j))])

### Mitigator

In [15]:
from qiskit.evaluators.backends import ReadoutErrorMitigation

backend = AerSimulator.from_backend(FakeBogota())
mit = ReadoutErrorMitigation(backend, mitigation="tensored", refresh=1, shots=2000, mit_pattern=[[0], [1]])
expval_raw = PauliExpectationValue(ansatz, observable, backend=backend)
expval_mit = PauliExpectationValue(ansatz, observable, backend=mit)
shots = 2000
print(f"w/o mitigation shots={shots}, result={expval_raw.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}")
print(f"w/  mitigation shots={shots}, result={expval_mit.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}")

w/o mitigation shots=2000, result=ExpectationValueResult(value=array(1.488+0.j), variance=array(12.100568+0.j), confidence_interval=[((1.3660514615773254+0j), (1.6099485384226742+0j))])
w/  mitigation shots=2000, result=ExpectationValueResult(value=array(1.76213738+0.j), variance=array(11.41804634+0.j), confidence_interval=[((1.6456821109309265+0j), (1.8785926501482777+0j))])
